In [7]:
from __future__ import division
import pyomo.environ as pyo
import numpy as np
# import polytope as pt
import matplotlib.pyplot as plt
import logging
from pyomo.util.infeasible import log_infeasible_constraints
# Set the logger to DEBUG level
logging.basicConfig(level=logging.DEBUG)

# GLOBAL VARIABLES
r_cm = 3   # cm 
L_cm = 5  # cm
Ts = 0.2   # s
MAX_deltaU = 1

def solve_cftoc_batch(q0, qf, N, r, L, Ts, MAX_deltaU):
    model = pyo.ConcreteModel()

    #Define the variables
    nq = 5
    nu = 2
    model.tidx = pyo.Set(initialize=range(N+1), ordered=True)
    model.tuidx = pyo.Set(initialize=range(N), ordered=True)
    model.qidx = pyo.Set(initialize=range(nq), ordered=True)
    model.uidx = pyo.Set(initialize=range(nu), ordered=True)

    model.q = pyo.Var(model.qidx, model.tidx)
    model.u = pyo.Var(model.uidx, model.tuidx, bounds=(-MAX_deltaU, MAX_deltaU))

    #Define the constraints

    #initial condition - fix these at the initial state
    for i in model.qidx:
        model.q[i, 0].fix(q0[i])

    #System dynamics, don't include the last time step, so use tuidx
    model.sys_dyn0 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.q[0, t+1] == model.q[0, t] + Ts*(r/2)*pyo.cos(model.q[2,t]) * (model.u[0, t] + model.u[1, t]))
    model.sys_dyn1 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.q[1, t+1] == model.q[1, t] + Ts*(r/2)*pyo.sin(model.q[2,t]) * (model.u[0, t] + model.u[1, t]))
    model.sys_dyn2 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.q[2, t+1] == model.q[2, t] + Ts*(r/L)*(-model.u[0, t] + model.u[1, t]))
    model.sys_dyn3 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.q[3, t+1] == model.q[3, t] + Ts*model.u[0, t])
    model.sys_dyn4 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.q[4, t+1] == model.q[4, t] + Ts*model.u[1, t])
    
    #Final Constraint, leave x free
    epsilon = 0.01
    model.final_constraint1 = pyo.Constraint(expr = (model.q[1,N] - qf[1])**2 <= epsilon)
    model.final_constraint2 = pyo.Constraint(expr = (model.q[2,N] - qf[2])**2 <= epsilon)
    model.final_constraint3 = pyo.Constraint(expr = (model.q[3,N] - qf[3])**2 <= epsilon)
    model.final_constraint4 = pyo.Constraint(expr = (model.q[4,N] - qf[4])**2 <= epsilon)


    # Cost function
    def cost_rule(model):
        cost_q = 0
        cost_u = 0

        for t in model.tidx:
            cost_q += model.q[0,t]**2

        return cost_q
    
    model.cost = pyo.Objective(rule=cost_rule, sense=pyo.minimize)

    # Solve the optimization problem
    solver = pyo.SolverFactory('ipopt')
    results = solver.solve(model)
  
    feas = str(results.solver.termination_condition) == "optimal"
    qOpt = np.asarray([model.q[:,t]() for t in model.tidx]).T
    uOpt = np.asarray([model.u[:,t]() for t in model.tuidx]).T
    JOpt = model.cost()
    log_infeasible_constraints(model)

    return feas, qOpt, uOpt, JOpt, model, results


q0 = np.array([0, 0, 0, 0, np.pi/2])
qf = np.array([0, 0, 0, 0, 0])
N = 10
feas, qOpt, uOpt, JOpt, model, results = solve_cftoc_batch(q0, qf, N, r_cm, L_cm, Ts, MAX_deltaU)


DEBUG:pyomo.core:Constructing ConcreteModel 'ConcreteModel', from data=None
DEBUG:pyomo.core:Constructing AbstractOrderedScalarSet 'tidx' on [Model] from data=None
DEBUG:pyomo.core:Constructing Set, name=tidx, from data=None
DEBUG:pyomo.core:Constructed component ''[Model].tidx'':
tidx : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

DEBUG:pyomo.core:Constructing AbstractOrderedScalarSet 'tuidx' on [Model] from data=None
DEBUG:pyomo.core:Constructing Set, name=tuidx, from data=None
DEBUG:pyomo.core:Constructed component ''[Model].tuidx'':
tuidx : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

DEBUG:pyomo.core:Constructing AbstractOrderedScalarSet 'qidx' on [Model] from data=None
DEBUG:pyomo.core:Constructing Set, name=qidx, from data=None
DEBUG:pyomo.core:Constructed component ''[Mod